In [1]:
# Импорт всех необходимых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
print("Все библиотеки загружены!")

Все библиотеки загружены!


In [2]:
# Загрузка данных
df = pd.read_csv('../data/cleaned_cbb.csv')
print("Данные загружены. Пример:")
df.head(3)

Данные загружены. Пример:


,TEAM,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,TORD,...,conf_P12,conf_Pat,conf_SB,conf_SC,conf_SEC,conf_SWAC,conf_Slnd,conf_Sum,conf_WAC,conf_WCC
0,North Carolina,40,0.807692,0.849351,0.344937,0.9531,52.6,48.1,15.4,18.2,...,False,False,False,False,False,False,False,False,False,False
1,Wisconsin,40,0.923077,1.000000,0.303797,0.9758,54.8,47.7,12.4,15.8,...,False,False,False,False,False,False,False,False,False,False
2,Michigan,40,0.807692,0.618182,0.202532,0.9375,53.9,47.7,14.0,19.5,...,False,False,False,False,False,False,False,False,False,False


In [3]:
## 1. Очистка данных (из data_cleaning.py)

In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler


def clean_data(df):
    """Очистка данных"""
    df_clean = df.dropna().drop_duplicates()

    # Исправлено: 'COMF' → 'CONF' (верное название столбца)
    df_clean = pd.get_dummies(df_clean, columns=['CONF'], prefix='conf')

    # Исправлено: дублирование 'ADJDE' → 'ADJOE'
    numerical_cols = ['ADJOE', 'ADJDE', 'W']
    scaler = MinMaxScaler()
    df_clean[numerical_cols] = scaler.fit_transform(df_clean[numerical_cols])

    return df_clean


if __name__ == "__main__":
    input_path = "../data/cbb.csv"  # Две точки - подняться на уровень выше
    output_path = "../data/cleaned_cbb.csv"

    df = pd.read_csv(input_path)
    df_clean = clean_data(df)
    df_clean.to_csv(output_path, index=False)
    print("Данные успешно обработаны и сохранены!")

Данные успешно обработаны и сохранены!


## 2. Разведочный анализ (из eda_analysis.py)

In [5]:
# Импорт библиотек
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# 1. Загрузка очищенных данных
df = pd.read_csv('../data/cleaned_cbb.csv')

# 2. Создаем папку results/task2
os.makedirs('../results/task2', exist_ok=True)

# 3. Базовые статистики
numeric_cols = ['ADJOE', 'ADJDE', 'W', 'G']  # Числовые столбцы
stats = df[numeric_cols].describe()

# Добавляем моду
stats.loc['mode'] = df[numeric_cols].mode().iloc[0]

# 4. Сохранение статистик в CSV
stats.to_csv('../results/task2/basic_stats.csv')
print("Базовые статистики сохранены в ../results/task2/basic_stats.csv")

# 5. Визуализация
## Гистограмма для ADJOE
plt.figure(figsize=(10, 6))
sns.histplot(df['ADJOE'], kde=True, bins=20, color='skyblue')
plt.title('Распределение эффективности атаки (ADJOE)')
plt.savefig('../results/task2/adjoie_hist.png', bbox_inches='tight', dpi=300)
plt.close()

## Box-plot для ADJDE
plt.figure(figsize=(10, 6))
sns.boxplot(x=df['ADJDE'], color='lightgreen')
plt.title('Распределение эффективности защиты (ADJDE)')
plt.savefig('../results/task2/adjde_boxplot.png', bbox_inches='tight', dpi=300)
plt.close()

## Scatter plot: ADJOE vs W
plt.figure(figsize=(10, 6))
sns.scatterplot(x='ADJOE', y='W', data=df, alpha=0.6, color='coral')
plt.title('Зависимость побед (W) от эффективности атаки (ADJOE)')
plt.savefig('../results/task2/adjoie_vs_w.png', bbox_inches='tight', dpi=300)
plt.close()

## Тепловая карта корреляций
plt.figure(figsize=(12, 8))
sns.heatmap(
    df[numeric_cols].corr(),
    annot=True,
    cmap='coolwarm',
    center=0,
    fmt=".2f",
    linewidths=0.5
)
plt.title('Корреляция между признаками', pad=20)
plt.savefig('../results/task2/correlation_heatmap.png', bbox_inches='tight', dpi=300)
plt.close()

# 6. Дополнительная визуализация (pair plot)
plt.figure(figsize=(12, 8))
sns.pairplot(df[numeric_cols], corner=True, diag_kind='kde')
plt.suptitle('Парные распределения числовых признаков', y=1.02)
plt.savefig('../results/task2/pair_plot.png', bbox_inches='tight', dpi=300)
plt.close()

print("""
EDA выполнена! 
""")

Базовые статистики сохранены в ../results/task2/basic_stats.csv

EDA выполнена! 



<Figure size 1200x800 with 0 Axes>

## 3. Создание новых признаков (из feature_engineering.py)

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

# 1. Загрузка данных
df = pd.read_csv('../data/cleaned_cbb.csv')

# 2. Создаем папку results/task3
os.makedirs('../results/task3', exist_ok=True)

# 3. Создание новых признаков
## Отношение атаки к защите
df['OFF_DEF_RATIO'] = df['ADJOE'] / df['ADJDE']

## Группы по эффективности
df['EFFICIENCY_GROUP'] = pd.cut(df['ADJOE'],
                               bins=[0, 0.3, 0.6, 1.0],
                               labels=['Low', 'Medium', 'High'])

## Логарифм количества игр
df['LOG_GAMES'] = np.log1p(df['G'])

# 4. Визуализация новых признаков
## Гистограмма для OFF_DEF_RATIO
plt.figure(figsize=(10, 6))
sns.histplot(df['OFF_DEF_RATIO'], kde=True, bins=20, color='royalblue')
plt.title('Распределение OFF_DEF_RATIO', fontsize=14)
plt.xlabel('Соотношение атаки/защиты', fontsize=12)
plt.ylabel('Частота', fontsize=12)
plt.savefig('../results/task3/off_def_ratio_hist.png', bbox_inches='tight', dpi=300)
plt.close()

## Boxplot для групп эффективности
plt.figure(figsize=(10, 6))
sns.boxplot(
    x='EFFICIENCY_GROUP',
    y='W',
    data=df,
    palette='Set2',
    hue='EFFICIENCY_GROUP',
    legend=False
)
plt.title('Распределение побед по группам эффективности', fontsize=14)
plt.xlabel('Группа эффективности', fontsize=12)
plt.ylabel('Количество побед (W)', fontsize=12)
plt.savefig('../results/task3/efficiency_vs_wins.png', bbox_inches='tight', dpi=300)
plt.close()

## Тепловая карта корреляций
plt.figure(figsize=(12, 8))
new_features = ['OFF_DEF_RATIO', 'LOG_GAMES', 'W', 'ADJOE', 'ADJDE']
corr_matrix = df[new_features].corr()
sns.heatmap(
    corr_matrix,
    annot=True,
    cmap='coolwarm',
    center=0,
    fmt=".2f",
    linewidths=0.5,
    annot_kws={'size': 12}
)
plt.title('Корреляция новых признаков с ключевыми показателями', fontsize=14, pad=20)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.savefig('../results/task3/new_features_corr.png', bbox_inches='tight', dpi=300)
plt.close()

# 5. Дополнительная визуализация: Violin plot
plt.figure(figsize=(10, 6))
sns.violinplot(
    x='EFFICIENCY_GROUP',
    y='OFF_DEF_RATIO',
    data=df,
    palette='pastel',
    hue='EFFICIENCY_GROUP',
    legend=False
)
plt.title('Распределение OFF_DEF_RATIO по группам эффективности', fontsize=14)
plt.xlabel('Группа эффективности', fontsize=12)
plt.ylabel('Соотношение атаки/защиты', fontsize=12)
plt.savefig('../results/task3/violinplot_efficiency.png', bbox_inches='tight', dpi=300)
plt.close()

# 6. Сохранение данных с новыми признаками
df.to_csv('../data/engineered_cbb.csv', index=False)

print("""
Feature Engineering выполнена! 
""")


Feature Engineering выполнена! 



## 4. Визуализации (из visualization.py)

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Настройка стиля
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# 1. Загрузка данных
df = pd.read_csv('../data/engineered_cbb.csv')

# 2. Создание папки results/task4 (если её нет)
os.makedirs('../results/task4', exist_ok=True)

# ======================
# 3. График 1: Bar chart (сравнение категорий)
# ======================
plt.figure(figsize=(10, 6))
ax1 = sns.barplot(
    x='EFFICIENCY_GROUP',
    y='W',
    data=df,
    errorbar=None,  # Замена устаревшего ci=None
    hue='EFFICIENCY_GROUP',  # Добавлено для palette
    palette="viridis",
    legend=False
)
plt.title('Среднее количество побед по группам эффективности')
plt.xlabel('Группа эффективности')
plt.ylabel('Среднее количество побед (W)')
plt.savefig('../results/task4/barplot_efficiency_vs_wins.png', bbox_inches='tight')
plt.close()

# ======================
# 4. График 2: Heatmap (корреляции)
# ======================
plt.figure(figsize=(12, 8))
corr_matrix = df[['ADJOE', 'ADJDE', 'W', 'OFF_DEF_RATIO']].corr()
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", center=0, fmt=".2f")
plt.title('Корреляции между ключевыми признаками')
plt.savefig('../results/task4/heatmap_correlations.png', bbox_inches='tight')
plt.close()

# ======================
# 5. График 3: Violin plot (распределения)
# ======================
plt.figure(figsize=(10, 6))
sns.violinplot(
    x='EFFICIENCY_GROUP',
    y='OFF_DEF_RATIO',
    data=df,
    palette="Set2",
    hue='EFFICIENCY_GROUP',
    legend=False
)
plt.title('Распределение OFF_DEF_RATIO по группам эффективности')
plt.savefig('../results/task4/violinplot_efficiency.png', bbox_inches='tight')
plt.close()

print("Все графики успешно сохранены в ../results/task4/!")

Все графики успешно сохранены в ../results/task4/!
